In [2]:
%cd /content/drive/MyDrive/CS381V-project/

/content/drive/MyDrive/CS381V-project


In [15]:
import argparse
import pandas as pd
import numpy as np
import os



In [17]:
def order(df, metric, base_dir = "inference_results"):
    # 1) Save CSV with rows ordered by IoU (highest first)
    df_sorted = df.sort_values(metric, ascending=False)
    sorted_csv = os.path.join(base_dir, f"sort_by_{metric}.csv")
    df_sorted.to_csv(sorted_csv, index=False)
    print(f"Saved sorted CSV to: {sorted_csv}")

In [10]:
def compute_row_acc(row, pred_col="pred", gt_col="gt"):
  # df["pred_list"] = df["pred"].apply(lambda s: [int(x) for x in s.split()])
# df["gt_list"]   = df["gt"].apply(lambda s: [int(x) for x in s.split()])
    pred = [int(x) for x in row[pred_col].split()]
    gt = [int(x) for x in row[gt_col].split()]
    n = len(gt)
    matches = sum(int(p == g) for p, g in zip(pred, gt))
    return matches / n

In [11]:
def compute_row_iou(row, pred_col="pred", gt_col="gt", end_label=2):
  # df["pred_list"] = df["pred"].apply(lambda s: [int(x) for x in s.split()])
# df["gt_list"]   = df["gt"].apply(lambda s: [int(x) for x in s.split()])
    pred = np.array([int(x) for x in row[pred_col].split()])
    gt = np.array([int(x) for x in row[gt_col].split()])
    n = len(gt)

    pred_region = (pred == end_label)
    gt_region = (gt == end_label)

    intersection = (pred_region & gt_region).sum().item()
    union = (pred_region | gt_region).sum().item()

    if gt_region.any():
        return True, intersection / union
    else:
        return False, 0.0

In [12]:
df = pd.read_csv("inference_results/LLM.csv")

# 1) add accuracy column
df["acc"] = df.apply(compute_row_acc, axis=1)

# 2) add IoU and IoU-valid columns
df[["iou_valid", "iou"]] = df.apply(
    lambda row: pd.Series(compute_row_iou(row)),
    axis=1
)

# 3) print average accuracy
print("Average accuracy:", df["acc"].mean())

# 4) save to CSV
df.to_csv("inference_results/with_metrics.csv", index=False)

Average accuracy: 0.515441293266419


In [22]:
df_metric = pd.read_csv("inference_results/with_metrics.csv")
df_data = pd.read_csv("HowToChangeData/annotation/test.csv")
df_merged = df_metric.merge(
    df_data[["video_name", "osc", "duration", "is_novel_osc"]],
    on="video_name",
    how="inner",    # or "left" since you said it's 1-1 and complete
)

# select & order columns
df_out = df_merged[
    ["video_name", "osc", "acc", "iou_valid", "iou", "pred", "gt", "duration", "is_novel_osc"]
]

# optional: inspect
print(df_out.head())
df_out.to_csv("inference_results/merged.csv", index=False)

                    video_name               osc    acc  iou_valid       iou  \
0  WOQxVNAVwGk_st253.0_dur40.0     rolling_pasta  0.600       True  0.588235   
1  k1rMwIt3hYo_st361.0_dur40.0     rolling_pasta  0.650       True  0.478261   
2  lUBEmi0kJ34_st376.0_dur40.0  squeezing_garlic  0.525       True  0.470588   
3  tEc1P55G0I8_st218.0_dur40.0  squeezing_garlic  0.775       True  0.894737   
4   VYkqjHmGumk_st77.0_dur40.0    mashing_butter  0.425      False  0.000000   

                                                pred  \
0  0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 ...   
1  0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 ...   
2  0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 ...   
3  0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 ...   
4  0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 2 ...   

                                                  gt  duration  is_novel_osc  
0  1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 ...      40.0         False  
1  1 1 1 1 1 1 1 1 1 1 1

In [23]:

df = pd.read_csv("inference_results/merged.csv")

order(df, "iou")
order(df, "acc")
# def print_best_worst_osc(input_csv: str):
#     df = pd.read_csv(input_csv)

#     # 2) Compute average IoU per OSC
#     osc_iou_mean = df.groupby("osc")["iou"].mean()

#     # Sort by mean IoU
#     osc_iou_sorted = osc_iou_mean.sort_values(ascending=False)

#     # Top 5 and bottom 5 (handles case with < 5 OSCs automatically)
#     top_5 = osc_iou_sorted.head(10)
#     bottom_5 = osc_iou_sorted.tail(10)

#     print("\nTop 10 OSCs by average IoU:")
#     for osc, val in top_5.items():
#         print(f"  {osc}: {val:.6f}")

#     print("\nBottom 10 OSCs by average IoU:")
#     for osc, val in bottom_5.items():
#         print(f"  {osc}: {val:.6f}")

#         # 1) Compute per-row accuracy
#     df["accuracy"] = df.apply(
#         compute_row_accuracy, axis=1
#     )


#     # 2) Average accuracy per OSC
#     valid = df.dropna(subset=["accuracy"])
#     osc_acc = valid.groupby("osc")["accuracy"].mean()

#     # Sort by average accuracy
#     osc_acc_sorted = osc_acc.sort_values(ascending=False)

#     top5 = osc_acc_sorted.head(10)
#     bottom5 = osc_acc_sorted.tail(10)

#     print("\nTop 10 OSCs by average accuracy:")
#     for osc, acc in top5.items():
#         print(f"  {osc}: {acc:.4f}")

#     print("\nBottom 10 OSCs by average accuracy:")
#     for osc, acc in bottom5.items():
#         print(f"  {osc}: {acc:.4f}")


Saved sorted CSV to: inference_results/sort_by_iou.csv
Saved sorted CSV to: inference_results/sort_by_acc.csv


In [ ]:
order("LLM_inference_result.csv", "LLM_inference_result_sorted.csv")

Saved sorted CSV to: LLM_inference_result_sorted.csv


In [ ]:
print_best_worst_osc("LLM_inference_result_sorted.csv")


Top 10 OSCs by average IoU:
  mincing_scallion: 0.777778
  roasting_squash: 0.756757
  peeling_squash: 0.743243
  whipping_meringue: 0.736842
  rolling_pasta: 0.680556
  slicing_beef: 0.667793
  peeling_potato: 0.592105
  blending_almond: 0.578947
  chopping_apple: 0.564286
  chopping_olive: 0.555556

Bottom 10 OSCs by average IoU:
  crushing_oreo: 0.000000
  squeezing_ginger: 0.000000
  whipping_sugar: 0.000000
  whipping_cream: 0.000000
  slicing_orange: 0.000000
  chopping_pepper: 0.000000
  zesting_clementine: 0.000000
  blending_egg: 0.000000
  blending_cream: 0.000000
  blending_coconut: 0.000000

Top 10 OSCs by average accuracy:
  peeling_garlic: 0.7750
  mincing_scallion: 0.7000
  peeling_squash: 0.7000
  roasting_squash: 0.7000
  whipping_meringue: 0.7000
  browning_sugar: 0.6750
  slicing_beef: 0.6625
  rolling_pasta: 0.6125
  blending_almond: 0.6000
  peeling_egg: 0.6000

Bottom 10 OSCs by average accuracy:
  grilling_potato: 0.0000
  mashing_egg: 0.0000
  frying_onion: 0.0